In [ ]:
############################################################
#
# Copyright 2017, 2019, 2020, 2021, 2022, 2023, 2024 Mohammed El-Afifi
# This file is part of processorSim.
#
# processorSim is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as
# published by the Free Software Foundation, either version 3 of the
# License, or (at your option) any later version.
#
# processorSim is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public
# License along with processorSim.  If not, see
# <http://www.gnu.org/licenses/>.
#
# program:      processor simulator
#
# file:         unified memory.ipynb
#
# function:     unified memory example
#
# description:  demonstrates the use of the simulator utilities for
#               tracking instruction execution inside a unified memory
#               processor architecture
#
# author:       Mohammed El-Afifi (ME)
#
# environment:  Visual Studio Code 1.92.0, python 3.12.4, Fedora release
#               40 (Forty)
#
# notes:        This is a private program.
#
############################################################

In [ ]:
import itertools
import logging
import pprint
import sys

In [ ]:
sys.path.append("../src")
import hw_loading
import processor_utils
from processor_utils import units
from processor_utils.units import FuncUnit
import program_utils
import sim_services

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
capabilities = ["ALU", "MEM"]
fetch_unit, decode_unit, execute_unit, memory_unit, writeback_unit = (
    units.UnitModel(
        name, 1, capabilities, units.LockInfo(rd_lock, wr_lock), mem_acl
    )
    for name, rd_lock, wr_lock, mem_acl in [
        ("F", False, False, capabilities),
        ("D", True, False, []),
        ("X", False, False, []),
        ("M", False, False, capabilities),
        ("W", False, True, []),
    ]
)

In [ ]:
proc_desc = processor_utils.ProcessorDesc(
    [fetch_unit],
    [FuncUnit(writeback_unit, [memory_unit])],
    [],
    itertools.starmap(
        FuncUnit,
        [
            [memory_unit, [execute_unit]],
            [execute_unit, [decode_unit]],
            [decode_unit, [fetch_unit]],
        ],
    ),
)
hw_desc = hw_loading.HwDesc(
    proc_desc,
    processor_utils.load_isa(
        [("LW", "MEM"), ("ADD", "ALU")],
        processor_utils.get_abilities(proc_desc),
    ),
)

In [ ]:
prog = program_utils.compile_program(
    program_utils.read_program(
        ["LW R1, (R2)", "ADD R3, R4, R5", "ADD R6, R7, R8", "ADD R9, R10, R11"]
    ),
    hw_desc.isa,
)

In [ ]:
pprint.pp(sim_services.simulate(prog, sim_services.HwSpec(hw_desc.processor)))